In [1]:
import paragami
import autograd
import autograd.numpy as np

In [2]:
pattern = paragami.NumericArrayPattern(shape=(2, 3, 4), lb=0, ub=10)
x = pattern.empty(valid=False)
x_bool = pattern.empty_bool(False)
#np.full_like(x, False, dtype='bool')
print(x_bool)
x_bool[1, 2] = True
x_bool[1, 1] = True
pattern.flat_indices(x_bool, free=True)

[[[False False False False]
  [False False False False]
  [False False False False]]

 [[False False False False]
  [False False False False]
  [False False False False]]]


array([16, 17, 18, 19, 20, 21, 22, 23])

In [3]:
pattern.flatten(x_bool, free=False)

array([False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False,  True,  True,
        True,  True,  True,  True,  True,  True])

In [4]:
# it = np.nditer(x_bool, flags=['multi_index'])
# while not it.finished:
#     print("%s <%s>" % (it[0], it.multi_index), end=' ')
#     it.iternext()

import itertools
free_len = pattern.flat_length(free=True)
flat_len = pattern.flat_length(free=False)
manual_jac = np.zeros((free_len, flat_len))
x_bool = np.full_like(x, False, dtype='bool')
for ind in itertools.product(*(range(n) for n in x_bool.shape)):
    x_bool[ind] = True
    flat_ind = pattern.flat_indices(x_bool, free=False)
    free_ind = pattern.flat_indices(x_bool, free=True)
    manual_jac[np.ix_(flat_ind, free_ind)] = 1
    x_bool[ind] = False

# print(manual_jac)

def flat_to_free(flat_val):
    return pattern.flatten(pattern.fold(flat_val, free=False), free=True)

flat_to_free(np.ones(flat_len))

get_flat_to_free_jac = autograd.jacobian(flat_to_free)
# print(flat_to_free_jac(np.ones(flat_len)) != 0)
# print(manual_jac != 0)
flat_to_free_jac = get_flat_to_free_jac(np.ones(flat_len) + 0.1)

#print(np.all((manual_jac != 0) == (flat_to_free_jac(np.ones(flat_len)) != 0)))
it = np.nditer(flat_to_free_jac, flags=['multi_index'])
while not it.finished:
    if it[0] != 0:
        assert manual_jac[it.multi_index] != 0
    it.iternext()

# it = np.nditer(x_bool, flags=['multi_index'])
# while not it.finished:
#     print("%s <%s>" % (it[0], it.multi_index), end=' ')
#     it.iternext()


In [5]:
a = np.arange(6).reshape((2, 3))
print(a)
print(a.flatten())
print(a[0, 1])

[[0 1 2]
 [3 4 5]]
[0 1 2 3 4 5]
1


In [18]:
pattern = paragami.SimplexArrayPattern(4, (2, 3))
x_bool = pattern.empty_bool(False)
x_bool[1, 1, 1] = True
x_bool[0, 0, 1] = True

print(pattern.flat_indices(x_bool, free=True))

[ 0  1  2 12 13 14]


In [26]:
pattern_dict = paragami.PatternDict()
pattern_dict['array'] = paragami.NumericArrayPattern(shape=(2, 3))
pattern_dict['simplex'] = paragami.SimplexArrayPattern(3, array_shape=(2, 3))
pattern_dict['mat'] = paragami.PSDSymmetricMatrixPattern(size=3)
pattern_subdict = paragami.PatternDict()
pattern_subdict['array'] = paragami.NumericArrayPattern(shape=(2,))
pattern_dict['dict'] = pattern_subdict

ind_vec = np.arange(pattern_dict.flat_length(free=False))
pattern_dict.fold(ind_vec, free=False, validate=False)

TypeError: fold() got an unexpected keyword argument 'validate'